In [1]:
import pybamm
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
param = pybamm.ParameterValues("OKane2022")
print(param)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x0000015268152020>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x00000152681527A0>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x0000015268152700>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function 

In [3]:
param.search("max")

Maximum concentration in negative electrode [mol.m-3]	33133.0
Maximum concentration in positive electrode [mol.m-3]	63104.0


In [3]:
model = pybamm.lithium_ion.DFN()
param_del = model.default_parameter_values
print(param_del)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.137,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x0000024BB19FFC40>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x0000024BB19FFBA0>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.6095950

In [10]:
param_del.search("max")

Maximum concentration in negative electrode [mol.m-3]	24983.2619938437
Maximum concentration in positive electrode [mol.m-3]	51217.9257309275


In [27]:
def battery_model(parameters):
    # Creating a DFN model
    model = pybamm.lithium_ion.DFN()
    
    # Loading the "OKane2022" parameter set
    param = pybamm.ParameterValues("OKane2022")
    
    # Updating parameters
    param.update({
        "Negative electrode thickness [m]": parameters[0], #l_neg :Anode thickness 35-79e-6
        "Positive electrode thickness [m]": parameters[1], #l_pos :Cathode thickness 35-79e-6
        "Separator thickness [m]": parameters[2], #l_sep :Separator thickness 10-30e-6
        "Negative electrode active material volume fraction": parameters[3], 
        #epsilon_f_neg: Anode active material volume fraction 0.4-0.5
        "Positive electrode active material volume fraction": parameters[4],
        #epsilon_f_pos: Cathode active material volume fraction 0.35-0.5
        "Negative particle radius [m]": parameters[5], #Rp_neg: 1-11e-6
        "Positive particle radius [m]": parameters[6], #Rp_pos: 1-11e-6
        "Initial concentration in electrolyte [mol.m-3]": parameters[7], #c0: 1000-1200
        
    })
    # Creating a Casadi solver and setting options
    solver = pybamm.CasadiSolver(
        mode="safe", 
        atol=1e-1, 
        rtol=1e-1, 
        dt_max=0.01,  # Decrease the maximum time step
        max_step_decrease_count=2000  # Increase in step size decrease in frequency
    )
    # solver = pybamm.CasadiSolver(mode="safe", atol=1e-6, rtol=1e-6)
    # Creating the ScikitsOdeSolver solver and setting options
    # solver = pybamm.ScikitsOdeSolver(method="bdf", atol=1e-4, rtol=1e-4)
    
    # Creating a Simulation
    sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
    
   # running simulation
    try:
        sim.solve([0, 1800])  # Reduced simulation time range of 1800 seconds
    except pybamm.SolverError as e:
        print(f"Solver failed for parameters: {parameters} with error: {e}")
        return np.nan  # Returns NaN to continue processing other combinations of parameters
    
    # Extract simulation results
    solution = sim.solution
    voltage = solution["Terminal voltage [V]"].entries
    
    # Returns the voltage at the end of the simulation
    return voltage[-1]


In [28]:
# Defining the parameter space
problem = {
    'num_vars': 8,
    'names': ['Negative electrode thickness [m]', 
              'Positive electrode thickness [m]', 
              'Separator thickness [m]',
              'Negative electrode active material volume fraction',
              'Positive electrode active material volume fraction',
              'Negative particle radius [m]',
              'Positive particle radius [m]',
              'Initial concentration in electrolyte [mol.m-3]',],
    'bounds': [[35e-6, 50e-6],  # Narrowing down the parameters
               [35e-6, 50e-6],  
               [10e-6, 30e-6],
               [0.4, 0.5],
               [0.35, 0.5],
               [1e-6, 5e-6],
               [1e-6, 5e-6],
               [1e3, 1.2e3]     # Initial concentration in electrolyte
               ]       
}

In [29]:
# Generate a sample
"""原样本生产为1024，现缩减为256"""
param_values = saltelli.sample(problem, 128)

In [30]:
print(param_values)

[[3.51757812e-05 3.99804687e-05 1.58593750e-05 ... 3.70312500e-06
  4.45312500e-06 1.04921875e+03]
 [3.65820313e-05 3.99804687e-05 1.58593750e-05 ... 3.70312500e-06
  4.45312500e-06 1.04921875e+03]
 [3.51757812e-05 4.72460937e-05 1.58593750e-05 ... 3.70312500e-06
  4.45312500e-06 1.04921875e+03]
 ...
 [4.16210937e-05 4.03320312e-05 1.27343750e-05 ... 3.54687500e-06
  4.54687500e-06 1.10390625e+03]
 [4.16210937e-05 4.03320312e-05 1.27343750e-05 ... 3.54687500e-06
  3.45312500e-06 1.03515625e+03]
 [4.16210937e-05 4.03320312e-05 1.27343750e-05 ... 3.54687500e-06
  3.45312500e-06 1.10390625e+03]]


In [31]:
# Run the model and get the results
Y = []

for i, params in enumerate(param_values):
    try:
        result = battery_model(params)
        Y.append(result)
    except Exception as e:
        print(f"Error for sample {i} with parameters {params}: {e}")
        Y.append(np.nan)  # Returns NaN to continue processing other combinations of parameters

# Converting Y to a NumPy array and filtering NaN values
Y = np.array(Y)
valid_indices = ~np.isnan(Y)
Y_valid = Y[valid_indices]
param_values_valid = param_values[valid_indices]

At t = 16.8572, repeated recoverable residual errors.
At t = 7.67513, repeated recoverable residual errors.
At t = 3.08436, repeated recoverable residual errors.
At t = 0.788906 and h = 9.38643e-19, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.788505 and h = 2.42901e-19, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.214967 and h = 3.26204e-17, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.214186 and h = 4.173e-09, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0716265, repeated recoverable residual errors.
At t = 0.071603 and h = 1.1386e-20, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0356944 and h = 7.77282e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0176243, repeated recoverable residual errors.
At t = 0.00883602 and h = 1.01939e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 16.8572 and 

Solver failed for parameters: [4.71289062e-05 3.97460938e-05 2.49218750e-05 4.98828125e-01
 3.51757812e-01 3.01562500e-06 2.26562500e-06 1.06484375e+03] with error: Maximum number of decreased steps occurred at t=1191.0000000000002 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=0.0) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.


At t = 2.23448 and h = 3.22974e-17, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 2.25411, repeated recoverable residual errors.
At t = 2.25408 and h = 8.15509e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 2.25412, repeated recoverable residual errors.
At t = 1.10635 and h = 7.77282e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.532498 and h = 2.84649e-21, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.245549, repeated recoverable residual errors.
At t = 0.101763 and h = 1.63749e-19, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0303174 and h = 6.22184e-23, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0301499, repeated recoverable residual errors.
At t = 0.0124204 and h = 4.27562e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 9.19035 and h = 1.51453e-24, the corrector convergence failed repeat

Solver failed for parameters: [4.71289062e-05 3.97460938e-05 1.19531250e-05 4.98828125e-01
 3.51757812e-01 3.01562500e-06 2.26562500e-06 1.06484375e+03] with error: Maximum number of decreased steps occurred at t=1191.0000000000002 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=0.0) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.


At t = 9.19048 and h = 2.93012e-20, the corrector convergence failed repeatedly or with |h| = hmin.
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual funct

Solver failed for parameters: [4.71289062e-05 3.97460938e-05 2.49218750e-05 4.98828125e-01
 3.51757812e-01 2.45312500e-06 2.26562500e-06 1.06484375e+03] with error: Maximum number of decreased steps occurred at t=1191.0000000000002 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=0.0) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.


At t = 14.9495 and h = 7.69586e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 5.76755 and h = 1.27824e-10, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.17608 and h = 2.00502e-15, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.17679 and h = 1.04385e-15, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.029054, repeated recoverable residual errors.
At t = 0.0289989 and h = 1.8349e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0289899 and h = 1.24437e-22, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0290825 and h = 3.17454e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0289936 and h = 8.55123e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0289959 and h = 1.33613e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.0110939 and h = 4.27562e-12

Solver failed for parameters: [4.71289062e-05 3.97460938e-05 2.49218750e-05 4.98828125e-01
 3.51757812e-01 3.01562500e-06 2.26562500e-06 1.12421875e+03] with error: Maximum number of decreased steps occurred at t=1191.0000000000002 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=0.0) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.


At t = 13.1747 and h = 3.22788e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 3.99296 and h = 6.245e-20, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 3.99263 and h = 4.19732e-17, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 13.1746, repeated recoverable residual errors.
At t = 3.99301, repeated recoverable residual errors.
At t = 3.99232, repeated recoverable residual errors.
At t = 13.1748 and h = 8.86564e-16, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 3.9929 and h = 1.9984e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 3.99256 and h = 1.91025e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.69766, repeated recoverable residual errors.
At t = 0.549341 and h = 2.0877e-15, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.548789, repeated recoverable residual errors.
At t = 0.263042, repeated recover

In [32]:
# Perform sensitivity analyses
Si = sobol.analyze(problem, Y_valid, print_to_console=True)

RuntimeError: 
        Incorrect number of samples in model output file.
        Confirm that calc_second_order matches option used during sampling.

In [117]:
# Print results
print("Sobol Sensitivity Indices")
print("S1:", Si['S1'])
print("ST:", Si['ST'])
print("S2:", Si['S2'])

AttributeError: 'list' object has no attribute 'size'